Definizione di tutti i metodi usati. **AVVIARE PRIMA DI QUALSIASI ALTRO CODICE**

In [3]:
import json
import re
import time
import requests
from enum import Enum

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.exc import GeocoderTimedOut, GeocoderQuotaExceeded

from sklearn.neighbors import BallTree
import numpy as np
import pandas as pd


with open('CL_ITTER107.json', 'r') as file:
    data = json.load(file)

# Estrai la lista di codici dal file JSON
codes = data['data']['codelists'][0]['codes']

# Associazioni per nomi non corrispondenti
associazioni_comune= {
    "Bolzano": "Bolzano/Bozen",
    "Bolzano / Bozen": "Bolzano/Bozen",
    "Forli": "Forlì"
}
# Funzione per trovare l'id in base al nome del comune
def find_id_comune_by_name(name):
    result= []
    for code in codes:
        if code['name']['it'] == name and re.match("^\d{6}$", code['id']):
            result.append(code['id'])

    if len(result) == 0 and name in associazioni_comune:
      name= associazioni_comune[name]
      for code in codes:
        if code['name']['it'] == name and re.match("^\d{6}$", code['id']):
            result.append(code['id'])

    print(name, result)
    if len(result) == 0:
        return None
    elif len(result) == 1:
        return result[0]
    return result

# Associazioni per nomi non corrispondenti
associazioni_regione_province= {
    "Provincia di Trento": "Trento",
    "Trentino-Alto Adige": "Trentino Alto Adige / Südtirol",
    "Bolzano": "Bolzano / Bozen",
    "Reggio Calabria": "Reggio di Calabria",
    "Provincia di Imperia": "Imperia"
}
# Funzione per trovare l'id in base al nome della regione o provincia
def find_id_regione_by_name(name):
    for code in codes:
        if code['name']['it'] == name and re.match("^IT.*$", code['id']):
            return code['id']

    if name in associazioni_regione_province:
      name= associazioni_regione_province[name]
      for code in codes:
        if code['name']['it'] == name and re.match("^IT.*$", code['id']):
            return code['id']
    else:
      return None
    return None

# Funzione per ottenere gli abitanti di un comume
def extract_abitanti_observation(response_json):
    try:
        observation = response_json['data']['dataSets'][0]['series']['0:0:0:0:0:0']['observations']['0'][0]
        return observation
    except KeyError:
        return None

# Funzione per ottenere i problemi in una regione
def extract_problemi_observation(response_json):
    try:
        observation = response_json['data']['dataSets'][0]['series']['0:0:0:0:0:0:0:0:0:0:0:0:0:0:0']['observations']['0'][0]
        return observation
    except KeyError:
        return None

# Funzione per ottenere le denunce in una regione
def extract_denunce_observation(response_json):
    try:
        observation = response_json['data']['dataSets'][0]['series']['0:0:0:0:0:0']['observations']['0'][0]
        return observation
    except KeyError:
        return None

# Funzione per ottenere la spesa media in una regione
def extract_spesamedia_observation(response_json):
    try:
        observation = response_json['data']['dataSets'][0]['series']['0:0:0:0:0:0:0:0:0:0:0']['observations']['0'][0]
        return observation
    except KeyError:
        return None

# Funzione per ottenere la superficie di un comune
def extract_superficie_observation(response_json):
    try:
        observation = response_json['data']['dataSets'][0]['series']['0:0:0']['observations']['0'][0]
        return observation
    except KeyError:
        return None

# Funzione per ottenere le aree a rischio idrogeologico in km2
def extract_rischi_idro_observation(response_json):
    try:
        observationHIGH = response_json['data']['dataSets'][0]['series']['0:0:0']['observations']['0'][0]
        observationLOW = response_json['data']['dataSets'][0]['series']['0:0:1']['observations']['0'][0]
        observationMED = response_json['data']['dataSets'][0]['series']['0:0:2']['observations']['0'][0]
        observations= [observationLOW, observationMED, observationHIGH]
        return observations
    except KeyError:
        return None


geolocator = Nominatim(user_agent="FindYourPlace")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, max_retries=5)
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1, max_retries=5)

# Funzione per ottenere location da un comune
def trova_location(comune):
    try:
      structuredQuery = {
        "postalcode" : '',
        "state" : '',
        "country" : 'Italia',
        "street" : '',
        "city" : {comune}
      }

      location = geocode(structuredQuery)
      if location is None:
        location = geocode(comune)
    except (GeocoderTimedOut, GeocoderQuotaExceeded, TimeoutError) as e:
      if (TimeoutError, GeocoderQuotaExceeded):
            print(e)
      else:
            print(f'Location not found: {e}')
            return None

    if location is None:
        print(f'Location for {comune} not found')
    return location

# Funzione per ottenere le coordinate di un comune
def trova_coordinate(comune):
    loc= trova_location(comune)

    if loc:
        latitudine, longitudine = loc.latitude, loc.longitude
        return latitudine, longitudine
    else:
        return None

# Funzione per ottenere il bounding box di un comune
def get_bounding_box(comune, bbox_type):
    valid = {"here", "overpass"}
    if bbox_type not in valid:
      raise ValueError("results: type must be one of %r." % valid)

    loc = trova_location(comune)

    if loc:
        bounding_box = loc.raw.get('boundingbox', None)
        if bounding_box:
            if bbox_type=="here":
              return bounding_box[2], bounding_box[0], bounding_box[3], bounding_box[1]
            elif bbox_type=="overpass":
              return bounding_box[0], bounding_box[2], bounding_box[1], bounding_box[3]

    return None

# Funzione per ottenere le coordinate più vicine in un dataframe
def trova_coordinate_vicine(latitudine, longitudine, dataframe):
    bt = BallTree(np.deg2rad(dataframe[['LAT', 'LON']].values), metric='haversine')
    distanze, indici = bt.query(np.deg2rad(np.c_[latitudine, longitudine]))

    nearest = dataframe.iloc[indici[0]]

    return nearest[['LAT', 'LON', 'Zona Sismica']]

# Funzione per ottenere la regione dal comune
def trova_regione_da_comune(comune):
    location = trova_location(comune)

    if location:
        position = reverse((location.latitude, location.longitude), language='it')
        regione = position.raw['address']['state']
        return regione
    else:
        return None

# Funzione per ottenere la regione dal comune
def trova_provincia_da_comune(comune):
    location = trova_location(comune)

    if location:
        position = reverse((location.latitude, location.longitude), language='it')
        try:
          regione = position.raw['address']['county']
        except:
          regione = position.raw['address']['province']
        return regione
    else:
        return None


### Lettura dati scaricati istat
class Request_Type(Enum):
    INQUINAMENTO = "inq"
    CRIMINALITA = "crim"
    DENUNCE = "den"
    SUPERFICIE= "sup"
    IDROGEOLOGICO= "idro"
    POPOLAZIONE= "pop"
    SPESA_MEDIA= "spesa"

# Funzione per trovare valore in locale invece che tramite api istat
def trova_valore_per_id(id, request_type):
  valid = {"inq", "crim", "den", "sup", "idro", "pop", "spesa"}
  if request_type not in valid:
      raise ValueError("Errore: request_type must be one of %r." % valid)

  if(request_type == "inq"):
    with open('istatData/inquinamentoRegioni.json', 'r') as file:
        jsonFile = json.load(file)
  elif(request_type == "crim"):
    with open('istatData/criminalitaRegioni.json', 'r') as file:
        jsonFile = json.load(file)
  elif(request_type == "den"):
    with open('istatData/denunceProvince.json', 'r') as file:
        jsonFile = json.load(file)
  elif(request_type == "sup"):
    with open('istatData/superficieComuni.json', 'r') as file:
        jsonFile = json.load(file)
  elif(request_type == "idro"):
    with open('istatData/rischioIdrogeologicoComuni.json', 'r') as file:
        jsonFile = json.load(file)
  elif(request_type == "pop"):
    with open('istatData/popolazione.json', 'r') as file:
        jsonFile = json.load(file)
  elif(request_type == "spesa"):
    with open('istatData/spesaMediaRegioni.json', 'r') as file:
        jsonFile = json.load(file)

  result= []
  for series in jsonFile['message:GenericData']['message:DataSet']['generic:Series']:
      series_key = series['generic:SeriesKey']['generic:Value']

      if type(id) is not list:
        ref_area_dict = next((i for i in series_key if i['@id'] in ['REF_AREA', 'ITTER107'] and i['@value'] == id), None)

        if ref_area_dict:
          try:
            result.append(float(series['generic:Obs']['generic:ObsValue']['@value']))
          except KeyError:
            result.append(-1)
      else:
        for id_x in id:
          ref_area_dict = next((i for i in series_key if i['@id'] in ['REF_AREA', 'ITTER107'] and i['@value'] == id_x), None)

          if ref_area_dict:
            try:
              result.append(float(series['generic:Obs']['generic:ObsValue']['@value']))
            except KeyError:
              result.append(-1)
            continue

  print(id, result)
  if len(result) == 0:
    return -1
  elif len(result) == 1:
    return result[0]
  return result


>Trova popolazione in base al nome del comune

In [ ]:
# Trova id tramite nome comune
comune_to_find = 'Salerno'
id_comune_found = find_id_comune_by_name(comune_to_find)

if id_comune_found:
    print(f"L'id corrispondente al nome '{comune_to_find}' è: {id_comune_found}")
else:
    print(f"Nome '{comune_to_find}' non trovato.")

# Costruisci l'URL con l'id trovato
api_url = f'https://sdmx.istat.it/SDMXWS/rest/data/22_289/.TOTAL.{id_comune_found}.9.99.?startPeriod=2023&format=jsondata'

# Effettua la richiesta all'API
response = requests.get(api_url)

# Verifica se la richiesta ha avuto successo (status code 200)
if response.status_code == 200:
    # Estrai l'observation dalla risposta JSON
    observation = extract_abitanti_observation(response.json())

    if observation is not None:
        print(f"L'observation della popolazione estratto è: {observation}")
    else:
        print("Observation della popolazione non trovato nella risposta JSON.")
else:
    print(f"Errore nella richiesta all'API. Codice di stato: {response.status_code}")


L'id corrispondente al nome 'Salerno' è: 065116
L'observation estratto è: 127485


Trova spesa media per consumi

In [ ]:
# Trova id tramite nome regione oppure Nord-Centro-Sud
regione_to_find = 'Campania'
id_regione_found = find_id_regione_by_name(regione_to_find)

if id_regione_found:
    print(f"L'id corrispondente al nome '{regione_to_find}' è: {id_regione_found}")
else:
    print(f"Nome '{regione_to_find}' non trovato.")

# Costruisci l'URL con l'id trovato
api_url = f'https://sdmx.istat.it/SDMXWS/rest/data/31_739/A.HH.99.SPESA_MEDIA..ALL..{id_regione_found}.ALL.ALL.TOT?startPeriod=2021&format=jsondata'

# Effettua la richiesta all'API
response = requests.get(api_url)

# Verifica se la richiesta ha avuto successo (status code 200)
if response.status_code == 200:
    # Estrai l'observation dalla risposta JSON
    observation = extract_spesamedia_observation(response.json())

    if observation is not None:
        print(f"L'observation della spesa media estratto è: {observation}")

        costo_vita= None
        if observation < 2100:
          costo_vita= "BASSO"
        elif observation > 2700:
          costo_vita= "ALTO"
        else:
          costo_vita= "MEDIO"
        print(f"Il Costo della Vita corrisponde a: {costo_vita}")
    else:
        print(f"Observation della spesa media non trovato nella risposta JSON.")

else:
    print(f"Errore nella richiesta all'API. Codice di stato: {response.status_code}")


L'id corrispondente al nome 'Campania' è: ITF3
L'observation della spesa media estratto è: 2034.65
Il Costo della Vita corrisponde a: BASSO


Trova presenza di problemi nella zona di residenza delle famiglie, come inquinamento, rumori o criminalità.<br>
Trova numero denunce.

In [ ]:
comune= 'Legnano'

# Trova id tramite nome regione oppure Nord-Centro-Sud
regione_to_find = trova_regione_da_comune(comune)
id_regione_found = find_id_regione_by_name(regione_to_find)

if id_regione_found:
    print(f"L'id corrispondente al nome '{regione_to_find}' è: {id_regione_found}")
else:
    print(f"Nome '{regione_to_find}' non trovato.")

class Problema(Enum):
    INQUINAMENTO = 1
    RUMORI = 2
    CRIMINALITA = 3

problema_scelto= Problema.INQUINAMENTO

# Costruisci l'URL con l'id trovato
api_url = f'https://sdmx.istat.it/SDMXWS/rest/data/33_291/A.{id_regione_found}...{problema_scelto.value}..........?startPeriod=2022&format=jsondata'

# Effettua la richiesta all'API
response = requests.get(api_url)

# Verifica se la richiesta ha avuto successo (status code 200)
if response.status_code == 200:
    # Estrai l'observation dalla risposta JSON
    observation = extract_problemi_observation(response.json())

    if observation is not None:
        print(f"L'observation di {problema_scelto.name} estratto è: {observation}")
    else:
        print(f"Observation di {problema_scelto.name} non trovato nella risposta JSON.")
else:
    print(f"Errore nella richiesta all'API. Codice di stato: {response.status_code}")


#### Segnalazioni relative a persone denunciate e arresti
provincia= trova_provincia_da_comune(comune)
id_provincia= find_id_regione_by_name(provincia)
print(f"L'id della provincia '{provincia}' è: {id_provincia}")

# Costruisci l'URL con l'id trovato
api_url = f'https://sdmx.istat.it/SDMXWS/rest/data/73_67/A.{id_provincia}.CRIMEN.TOT.9..?startPeriod=2022&format=jsondata'

# Effettua la richiesta all'API
response = requests.get(api_url)

# Verifica se la richiesta ha avuto successo (status code 200)
if response.status_code == 200:
    # Estrai l'observation dalla risposta JSON
    observation = extract_denunce_observation(response.json())

    if observation is not None:
        print(f"L'observation delle denunce estratto è: {observation}")
    else:
        print(f"Observation delle denunce non trovato nella risposta JSON.")
else:
    print(f"Errore nella richiesta all'API. Codice di stato: {response.status_code}")


L'id corrispondente al nome 'Lombardia' è: ITC4
L'observation di INQUINAMENTO estratto è: 12.9
L'id della provincia 'Milano' è: ITC45
L'observation delle denunce estratto è: 225078


Trova aree a pericolosità idrogeologica in km2

In [ ]:
# Trova id tramite nome comune
comune_to_find = 'Venezia'
id_comune_found = find_id_comune_by_name(comune_to_find)

if id_comune_found:
    print(f"L'id corrispondente al nome '{comune_to_find}' è: {id_comune_found}")
else:
    print(f"Nome '{comune_to_find}' non trovato.")

####Superficie di un comune
# Costruisci l'URL con l'id trovato
api_url = f'https://sdmx.istat.it/SDMXWS/rest/data/729_1050/A.{id_comune_found}.TOTAREA?startPeriod=2023&format=jsondata'

# Effettua la richiesta all'API
response = requests.get(api_url)

superficie= None
# Verifica se la richiesta ha avuto successo (status code 200)
if response.status_code == 200:
    # Estrai l'observation dalla risposta JSON
    observation = extract_superficie_observation(response.json())

    if observation is not None:
        print(f"L'observation della superficie estratto è: {observation}")
        superficie= observation/100
    else:
        print(f"Observation della superficie non trovato nella risposta JSON.")

else:
    print(f"Errore nella richiesta all'API. Codice di stato: {response.status_code}")


####Rischio idrogeologico in km2
# Costruisci l'URL con l'id trovato
api_url = f'https://sdmx.istat.it/SDMXWS/rest/data/729_995/A.{id_comune_found}.LOWFLHZ+MEDFLHZ+HIGHFLHZ?startPeriod=2017&format=jsondata'

# Effettua la richiesta all'API
response = requests.get(api_url)

rischio_idro= None
# Verifica se la richiesta ha avuto successo (status code 200)
if response.status_code == 200:
    # Estrai l'observation dalla risposta JSON
    observations = extract_rischi_idro_observation(response.json())

    if observations is not None:
        print(f"L'observationLOW estratto è: {observations[0]}")
        print(f"L'observationMED estratto è: {observations[1]}")
        print(f"L'observationHIGH estratto è: {observations[2]}")

        idrogeologicoLow= observations[0]-observations[1] # Rimuovo il Med (in cui vi è anche l'high)
        idrogeologicoMed= observations[1]-observations[2] # Rimuovo l' High
        idrogeologicoHigh= observations[2]

        rischio_ponderato= idrogeologicoLow*0.5+idrogeologicoMed*1+idrogeologicoHigh*1.5
        rischio_idro_perc= rischio_ponderato/max(superficie, rischio_ponderato)*100
        print(f"Il rischio idrogeologico è: {rischio_idro_perc}")

    else:
        print(f"Observation non trovato nella risposta JSON.")

else:
    print(f"Errore nella richiesta all'API. Codice di stato: {response.status_code}")


L'id corrispondente al nome 'Venezia' è: 027042
L'observation della superficie estratto è: 41589.27
L'observationLOW estratto è: 128.4
L'observationMED estratto è: 24.97
L'observationHIGH estratto è: 12.11
Il rischio idrogeologico è: 39.78910906587204


Estrazione Zone Sismiche.

In [ ]:
# Caricamento dei dati
sismi = pd.read_csv('italiaMedianaSismi.csv', sep=';')

def convert_percent(x):
    return float(x.replace(",", "."))

sismi["ag"] = sismi["ag"].apply(convert_percent)
sismi["LON"] = sismi["LON"].apply(convert_percent)
sismi["LAT"] = sismi["LAT"].apply(convert_percent)

min_value = sismi["ag"].min()
max_value = sismi["ag"].max()
sismi["normalized_ag"] = 4 * (sismi["ag"] - min_value) / (max_value - min_value)

def calcola_rischio(mediana):
    if mediana >= 3.0:
        return 1
    elif 2.0 <= mediana < 3.0:
        return 2
    elif 1.0 <= mediana < 2.0:
        return 3
    else:
        return 4

#Maggiore il numero, più il rischio
sismi['Zona Sismica']= sismi['normalized_ag'].apply(calcola_rischio)

print(sismi[["LON", "LAT", "Zona Sismica"]])
sismi[["LON", "LAT", "Zona Sismica"]].to_csv("Zone Sismiche.csv", index=False)

            LON      LAT  Zona Sismica
0       12.0899  47.0710             4
1       12.1193  47.0713             4
2       12.1487  47.0716             4
3       12.1781  47.0719             4
4       12.2075  47.0723             4
...         ...      ...           ...
104560  12.6943  35.3374             4
104561  12.7187  35.3376             4
104562  12.7432  35.3378             4
104563  12.7676  35.3380             4
104564  12.7679  35.3179             4

[104565 rows x 3 columns]


Trova Zona Sismica di un comune

In [ ]:
comune_da_cercare = "Napoli"
coordinate = trova_coordinate(comune_da_cercare)

if coordinate:
    print(f"Le coordinate di {comune_da_cercare} sono: {coordinate}")
else:
    print(f"Coordinate non trovate per {comune_da_cercare}")

if coordinate:
    zone_sismiche = pd.read_csv('Zone Sismiche.csv')

    # Trova le coordinate più vicine al punto di riferimento
    near_coords = trova_coordinate_vicine(coordinate[0], coordinate[1], zone_sismiche)

    print(f"Le coordinate più vicine a {comune_da_cercare} sono:")
    print(near_coords)
else:
    print(f"Coordinate non trovate per {comune_da_cercare}")

Le coordinate di Napoli sono: (40.8358846, 14.2487679)
Le coordinate più vicine a Napoli sono:
           LAT      LON  Zona Sismica
60116  40.8455  14.2452             3


Codice per convertire xml in json

In [ ]:
!pip install xmltodict
import xmltodict

with open("file.xml") as xml_file:
    data_dict = xmltodict.parse(xml_file.read())

json_data = json.dumps(data_dict)
with open("file.json", "w") as json_file:
        json_file.write(json_data)

Trovo minimo e massimo delle denunce nelle regioni per normalizzare in seguito le denunce.

In [ ]:
listaIdRegioni= ['ITC1', 'ITC2', 'ITC3', 'ITC4',
                 'ITD1', 'ITD2', 'ITD3', 'ITD4', 'ITD5', 'ITDA',
                 'ITE1', 'ITE2', 'ITE3', 'ITE4',
                 'ITF1', 'ITF2', 'ITF3', 'ITF4', 'ITF5', 'ITF6',
                 'ITG1', 'ITG2']

listaProvince = [
    "Agrigento","Alessandria","Ancona","Arezzo","Ascoli Piceno","Asti","Avellino","Bari","Barletta-Andria-Trani",
    "Belluno","Benevento","Bergamo","Biella","Bologna","Bolzano / Bozen","Brescia","Brindisi","Cagliari","Caltanissetta",
    "Campobasso","Caserta","Catania","Catanzaro","Chieti","Como","Cosenza","Cremona","Crotone","Cuneo",
    "Enna","Fermo","Ferrara","Firenze","Foggia","Forlì-Cesena","Frosinone","Genova","Gorizia","Grosseto","Imperia","Isernia",
    "La Spezia","L'Aquila","Latina","Lecce","Lecco","Livorno","Lodi","Lucca","Macerata","Mantova","Massa-Carrara","Matera",
    "Messina","Milano","Modena","Monza e della Brianza","Napoli","Novara","Nuoro","Oristano","Padova","Palermo","Parma","Pavia",
    "Perugia","Pesaro e Urbino","Pescara","Piacenza","Pisa","Pistoia","Pordenone","Potenza","Prato","Ragusa","Ravenna",
    "Reggio di Calabria","Reggio nell'Emilia","Rieti","Rimini","Roma","Rovigo","Salerno","Sassari","Savona",
    "Siena","Siracusa","Sondrio","Sud Sardegna","Taranto","Teramo","Terni","Torino","Trapani","Trento","Treviso","Trieste","Udine",
    "Varese","Venezia","Verbano-Cusio-Ossola","Vercelli","Verona","Vibo Valentia","Vicenza","Viterbo"
]


min_denunce_per_ab = float('inf')
max_denunce_per_ab = float('-inf')
for provincia in listaProvince:
  id_provincia= find_id_regione_by_name(provincia)
  denunce= int(trova_valore_per_id(id_provincia, Request_Type.DENUNCE.value))
  popolazione= int(trova_valore_per_id(id_provincia, Request_Type.POPOLAZIONE.value))

  if(denunce == -1 or popolazione == -1):
    continue

  denunce_per_ab= denunce/popolazione*100000
  if denunce_per_ab < min_denunce_per_ab:
      min_denunce_per_ab = denunce_per_ab
  if denunce_per_ab > max_denunce_per_ab:
      max_denunce_per_ab = denunce_per_ab

print("Valore minimo delle denunce ogni 100.000 abitanti: ", min_denunce_per_ab)
print("Valore massimo delle denunce ogni 100.000 abitanti: ", max_denunce_per_ab)

Valore minimo delle denunce ogni 100.000 abitanti:  1721.6031930816562
Valore massimo delle denunce ogni 100.000 abitanti:  6972.66361958435


Calcolo pericolosità in percentuale di un comune

In [ ]:
def calc_pericolosita(comune):
  id_comune = find_id_comune_by_name(comune)
  print(f'ID Comune {comune} ISTAT: {id_comune}')

  provincia= trova_provincia_da_comune(comune)
  id_provincia = find_id_regione_by_name(provincia)
  print(f'ID Provincia {provincia} ISTAT: {id_provincia}')

  regione= trova_regione_da_comune(comune)
  id_regione = find_id_regione_by_name(regione)
  print(f'ID Regione {regione} ISTAT: {id_regione}')

  coordinate= trova_coordinate(comune)
  #print(f'Coordinate Comune: {coordinate}\n')

  #### Inquinamento Regione
  inquinamento= trova_valore_per_id(id_regione, Request_Type.INQUINAMENTO.value)
  #print(f'Inquinamento {regione} con id {id_regione}: {inquinamento}')

  #### Criminalità Regione
  criminalità= trova_valore_per_id(id_regione, Request_Type.CRIMINALITA.value)
  #print(f'Criminalità {regione} con id {id_regione}: {criminalità}')

  #### Denunce per 100000 Abitanti Provincia
  popolazione= int(trova_valore_per_id(id_provincia, Request_Type.POPOLAZIONE.value))
  denunce= int(trova_valore_per_id(id_provincia, Request_Type.DENUNCE.value))
  if (denunce != -1):
    denunce_per_ab= denunce/popolazione*100000
  else:
    denunce_per_ab= min_denunce_per_ab
  #print(f'Denunce ogni 100.000 abitanti in {provincia} con id {id_provincia}: {denunce_per_ab}')

  #### Rischio idrogeologico Comune
  if id_comune is not None:
    idrogeologicoValues= trova_valore_per_id(id_comune, Request_Type.IDROGEOLOGICO.value)
  else:
    idrogeologicoValues= trova_valore_per_id(id_regione, Request_Type.IDROGEOLOGICO.value)

  if (idrogeologicoValues != -1):
    ## L'ordine è HIGH - LOW - MED, in Low è incluso Med, in cui a sua volta è incluso High
    idrogeologicoLow= idrogeologicoValues[1]-idrogeologicoValues[2] # Rimuovo il Med (in cui vi è anche l'high)
    idrogeologicoMed= idrogeologicoValues[2]-idrogeologicoValues[0] # Rimuovo l' High
    idrogeologicoHigh= idrogeologicoValues[0]
    ## Trovo superficie comune e calcolo il rischio ponderato
    superficie= trova_valore_per_id(id_comune, Request_Type.SUPERFICIE.value)/100
    rischio_ponderato= idrogeologicoLow*0.5+idrogeologicoMed*1+idrogeologicoHigh*1.5
    rischio_idro_perc= rischio_ponderato/max(superficie, rischio_ponderato)*100
  else:
    rischio_idro_perc= 0
  #print(f'Rischio idrogeologico {comune} con id {id_comune}: {rischio_idro_perc}')

  #### Zona Sismica Coordinate
  near_coords = trova_coordinate_vicine(coordinate[0], coordinate[1], pd.read_csv('Zone Sismiche.csv'))
  zona_sismica = int(near_coords.iloc[0]['Zona Sismica'])
  #print(f'Zona Sismica {comune} con id {id_comune}: {zona_sismica}\n')

  #### Calcolo indice di pericolosità
  ## Normalizzo le denunce --- Avviare il codice per calcolare min e max prima!
  if (max_denunce_per_ab - min_denunce_per_ab) <= 0:
      den_normal= 0.0
  else:
      den_normal= (denunce_per_ab - min_denunce_per_ab)/(max_denunce_per_ab - min_denunce_per_ab)*100
  ## Normalizzo la zona sismica
  sismi_normal= (1-(zona_sismica - 1)/3)*100

  ## Pesi per il calcolo ponderato
  peso_inquinamento = 0.15
  peso_criminalita = 0.2
  peso_denunce = 0.3
  peso_rischio_idrogeologico = 0.1
  peso_zona_sismica = 0.25
  return (
          peso_inquinamento * (inquinamento if inquinamento != -1 else 0) +
          peso_criminalita * (criminalità if criminalità != -1 else 0) +
          peso_denunce * den_normal +
          peso_rischio_idrogeologico * rischio_idro_perc +
          peso_zona_sismica * sismi_normal
      )

#print(f'La pericolosità è: {calc_pericolosita("Trento")}')

Trova Bounding Box di un comune e ricerca numero negozi, ristoranti e scuole<br>
Purtroppo il limite di pois trovabili è 100, per cui mi limito a cercare in un raggio di X metri solo determinate categorie.

In [12]:
class Poi_Type(Enum):
    NEGOZIO = "neg"
    RISTORANTE = "rist"
    SCUOLA = "scuola"

### trova numero poi di un comune in un certo raggio (MAX 100)
def trova_numero_poi_herev7(comune, poi_type, raggio= None):
  valid = {"neg", "rist", "scuola"}
  if poi_type not in valid:
      raise ValueError("Errore: poi_type must be one of %r." % valid)
  if raggio is None:
    if (poi_type == 'neg'): raggio= 1000
    elif (poi_type == 'rist'): raggio= 200
    elif (poi_type == 'scuola'): raggio= 1000

  bounding_box = get_bounding_box(comune,"here")
  coords= trova_coordinate(comune)

  if not bounding_box:
      print(f"Bounding box non trovato per {comune}")

  API_KEY= "oY1k3tXVkAI8O68lu62eXTEkuOc7TQb6Pwn2S_ZCXKo"

  ### Trova Negozi
  if (poi_type == 'neg'):
    api_url = (f'https://geocode.search.hereapi.com/v1/browse?'
    f'at={coords[0]},{coords[1]}'
    #f'&in=bbox:{bounding_box[0]},{bounding_box[1]},{bounding_box[2]},{bounding_box[3]}'
    f'&in=circle:{coords[0]},{coords[1]};r={raggio}'
    #https://www.here.com/docs/bundle/places-search-api-developer-guide/page/topics/place_categories/category-600-shopping.html
    f'&categories=600-6000,600-6100,600-6200,600-6600-0000,600-6700-0000'
    f'&limit=100&apiKey={API_KEY}')

    response = requests.get(api_url)
    num= None
    if response.status_code == 200:
        num = len(response.json()["items"])
    else:
        print(f"Errore nella richiesta all'API. Codice di stato: {response.status_code}")
    return num

  ### Trova Ristoranti
  elif (poi_type == 'rist'):
    api_url = (f'https://geocode.search.hereapi.com/v1/browse?'
    f'at={coords[0]},{coords[1]}'
    #f'&in=bbox:{bounding_box[0]},{bounding_box[1]},{bounding_box[2]},{bounding_box[3]}'
    f'&in=circle:{coords[0]},{coords[1]};r={raggio}'
    #https://www.here.com/docs/bundle/places-search-api-developer-guide/page/topics/place_categories/category-600-shopping.html
    f'&categories=100-1000-0000'
    f'&limit=100&apiKey={API_KEY}')

    response = requests.get(api_url)
    num= None
    if response.status_code == 200:
        num = len(response.json()["items"])
    else:
        print(f"Errore nella richiesta all'API. Codice di stato: {response.status_code}")
    return num

  ### Trova Scuole
  elif (poi_type == 'scuola'):
    api_url = (f'https://geocode.search.hereapi.com/v1/browse?'
    f'at={coords[0]},{coords[1]}'
    #f'&in=bbox:{bounding_box[0]},{bounding_box[1]},{bounding_box[2]},{bounding_box[3]}'
    f'&in=circle:{coords[0]},{coords[1]};r={raggio}'
    #https://www.here.com/docs/bundle/places-search-api-developer-guide/page/topics/place_categories/category-600-shopping.html
    f'&categories=800-8200-0000'
    f'&limit=100&apiKey={API_KEY}')

    response = requests.get(api_url)
    num= None
    if response.status_code == 200:
        num = len(response.json()["items"])
    else:
        print(f"Errore nella richiesta all'API. Codice di stato: {response.status_code}")
    return num

result= trova_numero_poi_herev7('Castel San Giorgio', Poi_Type.SCUOLA.value)
print(f'Il numero è: {result}')

Il numero è: 1


In questo codice invece provo ad usare overpass, ma in caso di comuni con pochi dati, questo risulterà inaffidabile.<br>
Infatti spesso il codice soprastante anche usando solo un certo raggio e certe categorie, riesce a trovare più negozi in un certo paese.

In [9]:
def trova_numero_negozi_overpass(comune):
    bbox = get_bounding_box(comune,"overpass")
    bbox_query = f"{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}"

    overpass_endpoint = "https://overpass-api.de/api/interpreter"

    # Definisci la query Overpass
    query = f"""
    [out:json];
    (
        node["building"="school"]({bbox_query});
        way["building"="school"]({bbox_query});
        relation["building"="school"]({bbox_query});
    );
    out count;
    """

    # Parametri della richiesta
    params = {
        'data': query
    }

    # Effettua la richiesta
    response = requests.get(overpass_endpoint, params=params)

    # Estrai il numero di negozi dal risultato
    result = response.json()
    numero_negozi = result['elements'][0]['tags']['total']

    return numero_negozi

# Trova il numero di negozi del comune
numero_negozi_trovati = trova_numero_negozi_overpass('Milano')
print(f"Numero nel bbox specificato: {numero_negozi_trovati}")

Numero nel bbox specificato: 0


Creazione dataset IdQ Comuni con dati assegnati<br>
**ATTENZIONE: Il file iniziale usato è stato modificato a mano per far combaciare i comuni con le varie risorse disponibili**

In [13]:
import os.path

if not(os.path.isfile('IdQ_Com_Coords.csv')):
  idq_com= pd.read_csv('IdQ_Comuni.csv')
  idq_com[['Latitudine', 'Longitudine']] = idq_com['Comune'].apply(lambda comune: pd.Series(trova_coordinate(comune)))
  print(idq_com)
  idq_com[['Comune', 'Latitudine', 'Longitudine', 'IdQ']].to_csv("IdQ_Com_Coords.csv", index=False)

if not(os.path.isfile('IdQ_Com_Peric.csv')):
  idq_com_peric= pd.read_csv('IdQ_Com_Coords.csv')
  idq_com_peric['Pericolosità'] = idq_com_peric['Comune'].apply(lambda com: calc_pericolosita(com))
  print(idq_com_peric)
  idq_com_peric.to_csv("IdQ_Com_Peric.csv", index=False)

def costo_vita(valore):
    if valore < 2100:
        return "BASSO"
    elif valore > 2700:
        return "ALTO"
    else:
        return "MEDIO"
if not(os.path.isfile('IdQ_Com_Costo.csv')):
  idq_com_costo= pd.read_csv('IdQ_Com_Peric.csv')
  idq_com_costo['Costo Vita'] = idq_com_costo['Comune'].apply(lambda comune: costo_vita(
      trova_valore_per_id(
        find_id_regione_by_name(
          trova_regione_da_comune(comune)
        ), Request_Type.SPESA_MEDIA.value
      )
    )
  )
  print(idq_com_costo)
  idq_com_costo.to_csv("IdQ_Com_Costo.csv", index=False)

if not(os.path.isfile('IdQ_Com_Abit.csv')):
  idq_com_abit= pd.read_csv('IdQ_Com_Costo.csv')
  idq_com_abit['Abitanti'] = idq_com_abit['Comune'].apply(lambda comune: trova_valore_per_id(
        find_id_comune_by_name(comune), Request_Type.POPOLAZIONE.value)
  )
  print(idq_com_abit)
  idq_com_abit.to_csv("IdQ_Com_Abit.csv", index=False)

if not(os.path.isfile('IdQ_Com_Sup.csv')):
  idq_com_sup= pd.read_csv('IdQ_Com_Abit.csv')
  idq_com_sup['Superficie'] = idq_com_sup['Comune'].apply(lambda comune: trova_valore_per_id(
        find_id_comune_by_name(comune), Request_Type.SUPERFICIE.value)/100
  )
  print(idq_com_sup)
  idq_com_sup.to_csv("IdQ_Com_Sup.csv", index=False)

if not(os.path.isfile('IdQ_Com_AbitPerKm2.csv')):
  idq_com_AbitPerKm2= pd.read_csv('IdQ_Com_Sup.csv')
  idq_com_AbitPerKm2['Abitanti per Km2'] = idq_com_AbitPerKm2['Comune'].apply(lambda comune:
          idq_com_AbitPerKm2.loc[idq_com_AbitPerKm2['Comune'] == comune, 'Abitanti'].values[0] /
          idq_com_AbitPerKm2.loc[idq_com_AbitPerKm2['Comune'] == comune, 'Superficie'].values[0]
  )
  print(idq_com_AbitPerKm2)
  idq_com_AbitPerKm2.to_csv("IdQ_Com_AbitPerKm2.csv", index=False)


def stima_poi_totali(comune, poi_type, superficie, raggio= None):
  valid = {"neg", "rist", "scuola"}
  if poi_type not in valid:
      raise ValueError("Errore: poi_type must be one of %r." % valid)
  if raggio is None:
    if (poi_type == 'neg'): raggio= 1000
    elif (poi_type == 'rist'): raggio= 200
    elif (poi_type == 'scuola'): raggio= 1000

  num_poi= trova_numero_poi_herev7(comune, poi_type, raggio)
  if(num_poi == None or num_poi == 0):
    num_poi= 1
  area_ricerca= (np.pi * raggio*raggio)/(10**6)
  stima= int(superficie/area_ricerca * num_poi)
  print(comune, poi_type, num_poi, stima)
  return stima

if not(os.path.isfile('IdQ_Com_Negozi.csv')):
  idq_com_neg= pd.read_csv('IdQ_Com_AbitPerKm2.csv')
  idq_com_neg['Num Negozi'] = idq_com_neg['Comune'].apply(lambda comune:
      stima_poi_totali(comune,
                       Poi_Type.NEGOZIO.value,
                       idq_com_neg.loc[idq_com_neg['Comune'] == comune, 'Superficie'].values[0])
  )
  print(idq_com_neg)
  idq_com_neg.to_csv("IdQ_Com_Negozi.csv", index=False)

if not(os.path.isfile('IdQ_Com_Rist.csv')):
  idq_com_rist= pd.read_csv('IdQ_Com_Negozi.csv')
  idq_com_rist['Num Ristoranti'] = idq_com_rist['Comune'].apply(lambda comune:
      stima_poi_totali(comune,
                       Poi_Type.RISTORANTE.value,
                       idq_com_rist.loc[idq_com_rist['Comune'] == comune, 'Superficie'].values[0])
  )
  print(idq_com_rist)
  idq_com_rist.to_csv("IdQ_Com_Rist.csv", index=False)

if not(os.path.isfile('IdQ_Com_Final.csv')):
  idq_com_final= pd.read_csv('IdQ_Com_Rist.csv')
  idq_com_final['Num Scuole'] = idq_com_final['Comune'].apply(lambda comune:
      stima_poi_totali(comune,
                       Poi_Type.SCUOLA.value,
                       idq_com_final.loc[idq_com_final['Comune'] == comune, 'Superficie'].values[0])
  )
  print(idq_com_final)
  idq_com_final.to_csv("IdQ_Com_Final.csv", index=False)


Agrigento scuola 3 232
Varese scuola 7 122
Venezia scuola 6 794
Verbania scuola 3 35
Vercelli scuola 5 126
Verona scuola 21 1329
Vibo Valentia scuola 6 88
Vicenza scuola 8 205
Viterbo scuola 6 775
Belluno scuola 3 140
Benevento scuola 11 458
Bergamo scuola 21 268
Biella scuola 7 104
Bologna scuola 22 986
Bolzano / Bozen scuola 8 133
Brescia scuola 19 546
Brindisi scuola 7 742
Cagliari scuola 11 296
Alessandria scuola 13 842
Caltanissetta scuola 5 670
Campobasso scuola 7 125
Caserta scuola 1 17
Catania scuola 9 523
Catanzaro scuola 1 35
Chieti scuola 5 94
Como scuola 17 200
Cosenza scuola 17 204
Cremona scuola 11 246
Crotone scuola 7 405
Ancona scuola 8 317
Cuneo scuola 5 190
Enna scuola 4 456
Fermo scuola 1 39
Ferrara scuola 8 1031
Firenze scuola 42 1367
Foggia scuola 10 1621
Forlì scuola 10 726
Frosinone scuola 9 134
Genova scuola 29 2218
Gorizia scuola 11 144
Grosseto scuola 7 1055
Imperia scuola 1 14


Isernia scuola 7 154
L'Aquila scuola 1 150
La Spezia scuola 7 114
Latina scuola 10 883
Lecce scuola 10 761
Lecco scuola 7 100
Livorno scuola 11 366
Lodi scuola 10 131
Arezzo scuola 12 1469
Lucca scuola 11 650
Macerata scuola 6 176
Mantova scuola 11 223
Massa scuola 3 89
Matera scuola 10 1248
Messina scuola 10 680
Milano scuola 31 1792
Modena scuola 6 349
Monza scuola 15 157
Napoli scuola 16 605
Ascoli Piceno scuola 9 452
Novara scuola 13 426
Nuoro scuola 5 305
Oristano scuola 9 242
Padova scuola 17 503
Palermo scuola 2 102
Parma scuola 11 912
Pavia scuola 15 301
Perugia scuola 12 1717
Pesaro scuola 6 291
Pescara scuola 11 120
Asti scuola 9 433
Piacenza scuola 8 301
Pisa scuola 14 824
Pistoia scuola 11 827
Pordenone scuola 8 97
Potenza scuola 7 390
Prato scuola 12 371
Ragusa scuola 4 566
Ravenna scuola 8 1664
Reggio di Calabria scuola 11 836
Reggio nell'Emilia scuola 11 807
Avellino scuola 9 87
Rieti scuola 6 394
Rimini scuola 19 821
Roma scuola 22 9014
Rovigo scuola 10 346
Salerno scuo